In [1]:
from pprint import pprint
import jmespath
import requests
from bs4 import BeautifulSoup as BS4
from textwrap import dedent
import re

In [2]:
req = requests.get('https://www.selver.ee/')
soup = BS4(req.content, features='html.parser')
#esilehelt URL otsingu regex
url_class = re.compile(r'(item level0 nav.\w+.+\d)')
#Hinna ja toote otsingu regex
product_class = re.compile(r'(col-lg-3 \w+).+') 
product_price = re.compile(r'("price")')
price_and_qty = re.compile(r'\s+(\d+.+)')
#Leia kõik HTML Li tagilt mis hoiab URLi
find_tag = soup.find_all('li',{'class':url_class})
#leia URL'id ja lista toote nimekirja pikkus
get_urls = [i.find_next('a').get('href')+"/?limit=96&p=" for i in find_tag]

In [3]:
html_list = list()
list_of_products = list()
list_of_prices = list()
dict_of_products = dict()

In [4]:
#start of the urls and end of the URLS
for z in range(5,23):
    #Find product last page
    product_html = requests.get(get_urls[z])
    #puhasta ilusa seebiga
    soup = BS4(product_html.content, features='html.parser')
    try:
        find_product_last_page = soup.find('a',{'class':'last'}).text
    except AttributeError:
        find_product_last_page = 3
    
    for i in range(1,int(find_product_last_page)+1):
        product_html = requests.get(get_urls[z]+str(i))
        soup = BS4(product_html.content, features='html.parser')
        html_list.append(soup)  
        print(f"Page URL: {get_urls[z]+str(i)} Last page is: {find_product_last_page}")

    for y in range(len(html_list)):  
        find_product_group = html_list[y].find_all('li',{'class':product_class})
        find_product_group_prices = html_list[y].find_all('span',{'class':'price'})
        find_product_group_titles = [i.find_next('a').get('title') for i in find_product_group]
        prices = [dedent(i.text.replace(u'\xa0',' ')) for i in find_product_group_prices if len(i) > 1]
        for t,f in zip(find_product_group_titles,prices):
            dict_of_products.update({t:re.split(price_and_qty,f)[:2]})
          


Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=1 Last page is: 4
Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=2 Last page is: 4
Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=3 Last page is: 4
Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=4 Last page is: 4
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=1 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=2 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=3 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=4 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=5 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=6 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=7 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=8 Last page is: 9
Page URL: https://www.selver

Page URL: https://www.selver.ee/enesehooldustarbed/?limit=96&p=10 Last page is: 12
Page URL: https://www.selver.ee/enesehooldustarbed/?limit=96&p=11 Last page is: 12
Page URL: https://www.selver.ee/enesehooldustarbed/?limit=96&p=12 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=1 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=2 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=3 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=4 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=5 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=6 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=7 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=8 Last page is: 12
Page URL: https://www.selver.ee/majapid

In [6]:
from sqlalchemy import create_engine, insert, update, MetaData, Table, String, Column, Integer

In [33]:
table_conn = create_engine('sqlite:///selver_data.db', echo=True)
toode = [{'toode':x,'hind':y[0],'hind_kogus':y[1] if len(y) > 1 else 0} for x,y in dict_of_products.items()]

In [34]:
conn = table_conn.connect()
meta = MetaData(table_conn)
table = Table(
               'SelverKaks', meta,
               Column('id', Integer, primary_key=True, autoincrement=True),
               Column('toode', String),
               Column('hind', String),
               Column('hind_kogus', String),
)
meta.create_all()

conn.execute(table.insert(),toode)


2021-03-11 12:32:59,403 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-11 12:32:59,405 INFO sqlalchemy.engine.base.Engine ()
2021-03-11 12:32:59,406 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-11 12:32:59,408 INFO sqlalchemy.engine.base.Engine ()
2021-03-11 12:32:59,410 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SelverKaks")
2021-03-11 12:32:59,412 INFO sqlalchemy.engine.base.Engine ()
2021-03-11 12:32:59,414 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SelverKaks")
2021-03-11 12:32:59,416 INFO sqlalchemy.engine.base.Engine ()
2021-03-11 12:32:59,418 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "SelverKaks" (
	id INTEGER NOT NULL, 
	toode VARCHAR, 
	hind VARCHAR, 
	hind_kogus VARCHAR, 
	PRIMARY KEY (id)
)


2021-03-11 12:32:59,419 INFO sqlalchemy.engine.base.Engine ()
2021-03-11 12:32:59,426 INFO sqlalchemy.engine.base.Engine COMMIT
2021-0

In [32]:
for x,y in dict_of_products.items():
    if(len(y) == 1):
        print(x,y)

      

Sarapuupähklitega piimašokolaadipallid Lindor, LINDT, 200 g ['25,90 €    ']


In [19]:
conn.close()

In [91]:
test = re.compile(r'\s+(\d+.+)')
stringOne = '2,19 €  2,19 tk '
stringTwo = '1,39 € 1,39 €/kg'
prices = re.split(test,stringOne)
pricesTwo =  re.split(test,stringTwo)
print(prices)
print(pricesTwo)

['2,19 €', '2,19 tk ', '']
['1,39 €', '1,39 €/kg', '']
